## Imports

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
import seaborn as sn
import matplotlib.pyplot as plt

## Get data

In [ ]:
file_path = r""

data = pd.read_excel(file_path)

# change all "." in variable names to "__"
data = data.rename(columns=lambda x: x.replace(".", "__"))

# reduce dataset to reporter == "parent"
data = data.query("reporter == 'parent'")

sdq_vars = [
    'e_sdq.d00149_hyp_sum'
]
sdq_vars = [c.replace(".", "__") for c in sdq_vars]
print("Modified voice variable names:", sdq_vars)

voice_vars = [
    'stimme.f0_sprech_1',
    'stimme.f0_sprech_2',
    'stimme.f0_sprech_3',
    'stimme.f0_sprech_4',
    'stimme.f0_sprech_5',
    'stimme.spl_sprech_1',
    'stimme.spl_sprech_2',
    'stimme.spl_sprech_3',
    'stimme.spl_sprech_4',
    'stimme.spl_sprech_5',
    'stimme.mpt',
    'stimme.jitter',
    'stimme.dsi'
]
voice_vars = [c.replace(".", "__") for c in voice_vars]
print("Modified voice variable names:", voice_vars)

covariates = [
    'age', 
    'sex12', 
    'soz_winkler_2019.d00408_gesamt_score',
    'c_pub_stat.d00077_pub_status',
    'c_anthro_kh.d00040_bmi_sds', 
]
covariates = [c.replace(".", "__") for c in covariates]
print("Modified covariate names:", covariates)

# Assign baseline age 
data["age_baseline"] = data.groupby("pseudosic")["age"].transform("first")

# Select variables we are interested in
data = (
    data[["pseudosic", "visnr", "age_baseline", "sex", "jahr"] + covariates + sdq_vars + voice_vars] 
    .reset_index(drop=True)
)

data.head()

## Linear mixed model (Main analysis)

### LMM Main: Analysis

In [ ]:
# Specify SDQ variable
sdq_var = "e_sdq__d00149_hyp_sum"

# Empty dictionary for results
res_lmm = {}

# Iterate over voice variables
for voice_var in voice_vars:
    
    # Create formula
    formula = f"{sdq_var} ~ {voice_var} + age_baseline + sex12 + soz_winkler_2019__d00408_gesamt_score + c_anthro_kh__d00040_bmi_sds"
    print(formula)
    
    # Filter and preprocess data (incl. dropping rows with missing data)
    data_lmm = data[["pseudosic", "visnr", sdq_var, voice_var] + ["age_baseline", "sex12", "soz_winkler_2019__d00408_gesamt_score", "c_anthro_kh__d00040_bmi_sds"]].copy()
    data_lmm.dropna(inplace=True)
    
    # Z-scaling of variables
    vars_to_scale = [sdq_var, voice_var] + ["age_baseline", "sex12", "soz_winkler_2019__d00408_gesamt_score", "c_anthro_kh__d00040_bmi_sds"]
    data_lmm[vars_to_scale] = data_lmm[vars_to_scale].apply(lambda x: (x - x.mean()) / x.std())
    
    # Fit model
    model = smf.mixedlm(
        formula,
        data=data_lmm,
        groups="pseudosic",
        re_formula="~visnr",
    ).fit(method="powell", maxiter=1000)
    
    # Store results in a dictionary
    model_results = pd.DataFrame({  
        "model_var": list(model.params.index),
        "n": [model.nobs] * len(model.params),
        "converged": [model.converged] * len(model.params),
        "df_model": [model.df_modelwc] * len(model.params),
        "df_resid": [model.df_resid] * len(model.params),
        "coef": model.params.values,
        "se": model.bse.values,
        "ci_lower": model.conf_int()[0].values,
        "ci_upper": model.conf_int()[1].values,
        "p": model.pvalues.values,
    })
    
    # Store results in dictionary 
    res_lmm[(sdq_var, voice_var)] = model_results

# Convert dictionary to DataFrame
res_lmm_df = pd.concat(res_lmm)

# Ensure correct index names
res_lmm_df.index.names = ["sdq_var", "voice_var", "model_var"]

# Reset only "sdq_var" and "voice_var", keeping "model_var" intact
if "model_var" in res_lmm_df.index.names:
    res_lmm_df = res_lmm_df.reset_index(level=["sdq_var", "voice_var"], drop=False)

# Reorder columns to generate proper excel output
final_columns = ["sdq_var", "voice_var", "model_var", "n", "converged", "df_model", "df_resid", "coef", "se", "ci_lower", "ci_upper", "p"]
res_lmm_df = res_lmm_df[final_columns]

# Assign FDR-adjusted p-values
res_lmm_df["q"] = multipletests(res_lmm_df["p"], method="fdr_bh")[1]

# Save results to Excel
res_lmm_df.to_excel("all_results_with_q_values.xlsx", index=False)

### Voice Variables only 
# Filter rows for voice variables
voice_vars_only = res_lmm_df.query("model_var in @voice_vars")

# Reset index properly (ensuring a clean DataFrame)
voice_vars_only = voice_vars_only.reset_index(drop=True)

# Save results to Excel file
voice_vars_only.to_excel("voice_vars_results.xlsx", index=False)

# Filter significant results (q < 0.05)
significant_voice_vars = voice_vars_only.query("q < 0.05")

# Save significant results to Excel
significant_voice_vars.to_excel("significant_voice_vars_results.xlsx", index=False)


### LMM Main: Extract significant effects

In [ ]:
# Drop 'model_var' temporarily to avoid duplication issues
significant_voice_vars_reset = significant_voice_vars.drop(columns=["model_var"]).reset_index()

# Extract significant results based on FDR-corrected q-values
vars_sig_lmm = (
    significant_voice_vars_reset
    .query("q < 0.05")  # Filter rows where q < 0.05
    .sort_values("q")   # Sort by q-values for clarity
    [["sdq_var", "voice_var"]]  # Select relevant columns
    .values.tolist()    # Convert to a list of significant variable pairs
)

print("Significant Variables List:", vars_sig_lmm)

### LMM Main: Regression plot for FDR significant effects (voice variables)

In [ ]:
variable_name_mapping = {
    "e_sdq__d00149_hyp_sum": "SDQ_HI",
    "stimme__f0_sprech_2": "f0_conversation_II",
    "stimme__f0_sprech_3": "f0_presentation_III",
    "stimme__f0_sprech_4": "f0_loud_IV",
    "stimme__f0_sprech_5": "f0_quiet_V",
    "stimme__spl_sprech_1": "spl_quiet_I",
    "stimme__spl_sprech_2": "spl_conversation_II",
    "stimme__spl_sprech_3": "spl_presentation_III",
    "stimme__spl_sprech_4": "spl_loud_IV",
    "stimme__spl_sprech_5": "spl_quiet_V",
}

# Iterate over significant effects and create individual plots
for i, (y_var, x_var) in enumerate(vars_sig_lmm):
    print(f"Plotting {y_var} ~ {x_var}")
    
    # Get the renamed variables
    y_var_renamed = variable_name_mapping.get(y_var, y_var)
    x_var_renamed = variable_name_mapping.get(x_var, x_var)
    
    # Create a new figure for each plot
    fig, ax = plt.subplots(figsize=(6, 4))  
    
    # Create scatter plot for the dataset
    sn.scatterplot(
        x=data[x_var],
        y=data[y_var],
        color="black",  
        alpha=0.2,  
        ax=ax
    )
    
    # Add regression line
    sn.regplot(
        x=data[x_var],
        y=data[y_var],
        scatter=False,  
        ax=ax,
        color="darkred"
    )
    
    # Annotate the plot with statistical information
    match = significant_voice_vars.query(
        f"sdq_var == '{y_var}' and voice_var == '{x_var}'"
    )
    if not match.empty:
        coef = match["coef"].values[0]
        p_value = match["p"].values[0]
        q_value = match["q"].values[0]

        ax.annotate(
            f"coef. = {coef:.3f}\n"
            f"p = {p_value:.3f}\n"
            f"q = {q_value:.3f}",
            xy=(0.95, 0.05),  
            ha="right", va="bottom",  
            xycoords="axes fraction",  
            fontsize=11,
            bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white")
        )
    
    # Set title, x-axis, and y-axis labels using renamed variables
    ax.set_title(f"{y_var_renamed} ~ {x_var_renamed}", weight="semibold")
    ax.set_xlabel(x_var_renamed, fontsize=12)  
    ax.set_ylabel(y_var_renamed, fontsize=12)  

    # Adjust layout to avoid overlap
    fig.tight_layout()
    
    # Save each plot in the current directory using renamed variables
    plot_filename = f"{y_var_renamed}_vs_{x_var_renamed}.png"  # File name for each plot
    fig.savefig(plot_filename, format="png", dpi=300)  # Save the plot
    plt.close(fig)  # Close the figure to save memory



In [ ]:
# Specify SDQ variable
sdq_var = "e_sdq__d00149_hyp_sum"

# Empty dictionary for results
res_lmm_rob = {}

# Iterate over voice variables
for voice_var in voice_vars:
    
    # Create formula
    formula = f"{sdq_var} ~ {voice_var} + sex12 + soz_winkler_2019__d00408_gesamt_score + c_anthro_kh__d00040_bmi_sds + c_pub_stat__d00077_pub_status"
    print(formula)
    
    # Filter and preprocess data (incl. dropping rows with missing data)
    data_lmm = data[["pseudosic", "visnr", sdq_var, voice_var] + ["sex12", "soz_winkler_2019__d00408_gesamt_score", "c_anthro_kh__d00040_bmi_sds", "c_pub_stat__d00077_pub_status"]].copy()
    data_lmm.dropna(inplace=True)
    
    # Z-scaling of variables
    vars_to_scale = [sdq_var, voice_var] + ["sex12", "soz_winkler_2019__d00408_gesamt_score", "c_anthro_kh__d00040_bmi_sds", "c_pub_stat__d00077_pub_status"]
    data_lmm[vars_to_scale] = data_lmm[vars_to_scale].apply(lambda x: (x - x.mean()) / x.std())
    
    # Fit model
    model = smf.mixedlm(
        formula,
        data=data_lmm,
        groups="pseudosic",
        re_formula="~visnr",
    ).fit(method="powell", maxiter=1000)
    
    # Store results in a dictionary
    model_results = pd.DataFrame({  
        "model_var": list(model.params.index),
        "n": [model.nobs] * len(model.params),
        "converged": [model.converged] * len(model.params),
        "df_model": [model.df_modelwc] * len(model.params),
        "df_resid": [model.df_resid] * len(model.params),
        "coef": model.params.values,
        "se": model.bse.values,
        "ci_lower": model.conf_int()[0].values,
        "ci_upper": model.conf_int()[1].values,
        "p": model.pvalues.values,
    })
    
    # Store results in dictionary 
    res_lmm_rob[(sdq_var, voice_var)] = model_results

# Convert dictionary to DataFrame
res_lmm_rob_df = pd.concat(res_lmm_rob)

# Ensure correct index names
res_lmm_rob_df.index.names = ["sdq_var", "voice_var", "model_var"]

# Reset only "sdq_var" and "voice_var", keeping "model_var" intact
if "model_var" in res_lmm_rob_df.index.names:
    res_lmm_rob_df = res_lmm_rob_df.reset_index(level=["sdq_var", "voice_var"], drop=False)

# Reorder columns to generate proper excel output
final_columns = ["sdq_var", "voice_var", "model_var", "n", "converged", "df_model", "df_resid", "coef", "se", "ci_lower", "ci_upper", "p"]
res_lmm_rob_df = res_lmm_rob_df[final_columns]

# Assign FDR-adjusted p-values
res_lmm_rob_df["q"] = multipletests(res_lmm_rob_df["p"], method="fdr_bh")[1]

# Save results to Excel
res_lmm_rob_df.to_excel("lmm_rob_pubstat_all_results.xlsx", index=False)


### Voice Variables only 
# Filter rows for voice variables
voice_vars_only = res_lmm_rob_df.query("model_var in @voice_vars")

# Reset index properly (ensuring a clean DataFrame)
voice_vars_only = voice_vars_only.reset_index(drop=True)

# Save results to Excel file
voice_vars_only.to_excel("voice_vars_results_pubstat.xlsx", index=False)

# Filter significant results (q < 0.05)
significant_voice_vars = voice_vars_only.query("q < 0.05")

# Save significant results to Excel
significant_voice_vars.to_excel("significant_voice_vars_results_pubstat.xlsx", index=False)